In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler


from sklearn import preprocessing
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix


from sklearn.model_selection import StratifiedKFold


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Load the entire dataset into a pandas dataframe

df = pd.read_parquet('/nobackup/amimalik/bits/dataset/dimensions/4Q/working_dataset.parquet')
model_df = pd.read_csv('/nobackup/amimalik/bits/dataset/dimensions/4Q/sorted_failures.csv')

model_df = model_df.head(5)

In [ ]:
df.shape

In [ ]:
# Group the instances dataframe by serial number
grouped_instances = df.groupby('serial_number')

# Create an empty list to store the last 150 samples for each serial number
last_n = []

# Iterate over each group
for name, group in grouped_instances:
    # Get the last n (150) samples for the current serial number
    last_n_group = group.tail(30)
    # Append the last n samples to the list
    last_n.append(last_n_group)

# Concatenate the list of dataframes into a single dataframe
instance_last_n = pd.concat(last_n)

# Reset the index of the dataframe
instance_last_n.reset_index(drop=True, inplace=True)

# Drop the date column from the dataframe
# instance_last_n.drop(columns=['date'], inplace=True)

# Print the updated dataframe
instance_last_n.head()

In [ ]:
instance_last_n.shape

In [ ]:
ddf = instance_last_n.copy()
#reset the index and drop old index
ddf.reset_index(drop=True, inplace=True)
ddf.to_parquet("/nobackup/amimalik/bits/dataset/dimensions/4Q/working_dataset_time.parquet")

In [ ]:
def get_model_data(df, model):
    model_df = df[df['model'] == model]
    model_df = model_df.drop(columns=['model'])
    model_df.drop(columns=['serial_number'], inplace=True)
    return model_df


In [ ]:
def get_clean_data(df):
    test = df.isnull().sum()
    d_col = []
    for k in test.keys():
        if test[k] == df.shape[0]:
            d_col.append(k)
    df.drop(columns=d_col, inplace=True)
    # replace null by mean values
    for col in df.columns:
        df[col].fillna(df[col].mean(), inplace=True)
    # reset index
    df.reset_index(drop=True, inplace=True)
    return df

In [ ]:
for dev_models in model_df['model']:
    model_data = get_model_data(ddf, dev_models)
    model_data = get_clean_data(model_data)

In [ ]:
model_df

In [ ]:
dev_models = 'ST12000NM0008'
model_data = get_model_data(df, dev_models)
model_data = get_clean_data(model_data)

In [ ]:
model_data.tail()

In [ ]:
model_data.shape

In [ ]:
# Define a function that performs stratified cross-validation

def stratified_data_split(df):
    # Define the number of folds
    n_folds = 5

    # Create an instance of StratifiedKFold
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

    # Split the data into X and y
    X = df.drop(columns=['failure'])
    y = df['failure']

    # Perform stratified cross-validation
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        y_train = np.array(y_train).reshape(-1, 1)
        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
        y_test = np.array(y_test).reshape(-1, 1)
        X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))


        # print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
        yield X_train, y_train, X_test, y_test

In [ ]:
def model_fit(X_train, y_train, x_test, y_test, epochs=10, batch_size=32, learning_rate=0.001):
    validation_number = int(0.9 * X_train.shape[0])
    validation_X = X_train[validation_number:]
    validation_y = y_train[validation_number:]
    train_X = X_train[:validation_number]
    train_y = y_train[:validation_number]


    # Define the model
    model = Sequential()
    model.add(LSTM(100, activation='relu', input_shape=(1,9),return_sequences=True, dropout=0.25))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    # Fit the model
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(validation_X, validation_y), verbose=0)

    # summarize performance of the model
    scores = model.evaluate(train_X, train_y, verbose=0)
    print(model.metrics_names)
    print("model loss: %.2f%%" % (scores*100))

    # Use the trained model to predict on the test_X dataset
    predictions = model.predict(test_X)

    # Print the predictions
    # print(predictions)

    # Convert the predictions to binary values (0 or 1)
    binary_predictions = (predictions > 0.5).astype(int)

    # Calculate the evaluation metrics
    accuracy = accuracy_score(test_y, binary_predictions)
    precision = precision_score(test_y, binary_predictions)
    recall = recall_score(test_y, binary_predictions)
    f1 = f1_score(test_y, binary_predictions)
    roc_auc = roc_auc_score(test_y, binary_predictions)

    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    print("ROC AUC score:", roc_auc)


    # plot history
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend()
    plt.grid()
    plt.show()

    return model

In [ ]:
for train_X, train_y, test_X, test_y in stratified_data_split(model_data):
        #print(train_X.shape, train_y.shape, test_X.shape)
        #print(train_X.shape[0], train_X.shape[1])
        #print(np.isnan(train_X).sum())
        #print(np.isnan(train_y).sum())

        #scaler = StandardScaler()
        #X_train_scaled = scaler.fit_transform(train_X)
        #X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))

        #print(X_train_scaled.shape, train_y.shape, test_X.shape)

        model_fit(train_X, train_y, test_X, test_y, epochs=10, batch_size=32, learning_rate=0.001)

In [ ]:
# model_fit(X_train, y_train, x_test, y_test, epochs=10, batch_size=32)
epochs=10
batch_size=32
learning_rate=0.001


# train_X, train_y, test_X, test_y
X_train = train_X
y_train = train_y

validation_number = int(0.9 * X_train.shape[0])
validation_X = X_train[validation_number:]
validation_y = y_train[validation_number:]

X_train = X_train[:validation_number]
y_train = y_train[:validation_number]


# Define the model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(1,9),return_sequences=True, dropout=0.25))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

# Fit the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(validation_X, validation_y), verbose=2)

# summarize performance of the model
scores = model.evaluate(train_X, train_y, verbose=2)
print(model.metrics_names)
print("model loss: %.2f%%" % (scores*100))

# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.grid()


In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.grid()

In [ ]:
# Use the trained model to predict on the test_X dataset
predictions = model.predict(test_X)

# Print the predictions
print(predictions)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert the predictions to binary values (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

# Calculate the evaluation metrics
accuracy = accuracy_score(test_y, binary_predictions)
precision = precision_score(test_y, binary_predictions)
recall = recall_score(test_y, binary_predictions)
f1 = f1_score(test_y, binary_predictions)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
count_ones = np.count_nonzero(test_y == 1)
print(count_ones)

In [ ]:
history.history

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()